# III. Основные инструменты геоанализа


В этом занятии мы познакомимся с базовыми методами пространственного анализа данных, используя возможности библиотек GeoPandas и Shapely. Мы научимся строить буферные зоны, выполнять пространственные объединения (spatial join), и пространственную фильтрацию, а также применять эти инструменты на практике: агрегировать точечные данные по ячейкам регулярной сетки, а также агрегировать их атрибутивную информацию

Эти навыки подготовят нас к анализу исторической вспышки холеры в Лондоне (1854) – [второму проекту](../projects/project2.ipynb) на нашем курсе.


**Импортируем библиотеки**


Первым делом подключим необходимые библиотеки


In [ ]:
import pandas as pd  
import geopandas as gpd
import osmnx as ox

from shapely.geometry import Point, Polygon, box
import matplotlib.pyplot as plt
import matplotlib as mpl   


<p style="color:#26528C; font-style:italic"><strong>GeoPandas</strong> – надстройка над pandas, расширяющая возможности DataFrame для работы с геометрией. Хранит объекты геометрии (точки, линии, полигоны) в специальном столбце `geometry` и предоставляет методы для пространственных операций (буферизация, объединение, объединение по пространственному признаку).</p> <p style="color:#26528C; font-style:italic"><strong>osmnx</strong> – библиотека для загрузки и анализа данных OpenStreetMap. Позволяет легко получать графы улиц, точки интереса и другие геоданные напрямую из OSM, а также выполнять маршрутизацию, визуализацию и базовый пространственный анализ городских сетей.</p><p style="color:#26528C; font-style:italic"><strong>Shapely</strong> – библиотека для создания и операций над геометрическими примитивами (точки, линии, полигоны). Предоставляет методы для объединения, пересечения, измерения расстояний и вычисления буферных зон.</p> <p style="color:#26528C; font-style:italic"><strong>matplotlib.pyplot</strong> – модуль визуализации, вдохновлённый MATLAB. Позволяет строить 2D-графики (линии, гистограммы, scatter-плоты), настраивать оформление, добавлять подписи, легенды и работать с осями.</p> <p style="color:#26528C; font-style:italic"><strong>contextily</strong> – утилита для подложек карт (tile basemaps) в matplotlib. Позволяет легко добавить тайловые слои (OpenStreetMap, Stamen и др.) под ваши геоплоты, автоматически приводя проекции в соответствие.</p>


## 1. Базовые инструменты пространственного анализа


Прежде чем переходить к основным инструментам, рассмотрим простые операции с геоданными: повторим как мы работаем с разными системами координат, рассмотрим измерение расстояний, буферизацию и пространственные отношения между объектами


### 1.1 Создание точек и перепроецирование (повторение)


Для дальнешей работы создадим две точки на основе координат в Санкт-Петербурге


In [ ]:
# Задаем координаты двух точек (долгота, широта) 
lon1, lat1 = 30.3359, 59.9316  # Основное здание Александринского театра (историческая сцена)
lon2, lat2 = 30.3322, 59.9385   # Государственный Русский музей (Михайловский дворец)

# Создаем список точек
geometry = [Point(lon1, lat1), Point(lon2, lat2)]

# Создаем GeoDataFrame с названиями и геометрией
points = gpd.GeoDataFrame({
    "name": ["Александринский театр", "Русский музей"],
    "geometry": geometry
}, crs="EPSG:4326")


Посмотрим на точки на карте


In [ ]:
points.explore(tiles='CartoDB positron')

Мы создали две точки в географической системе координат WGS84 – широта и долгота в градусах.

Однако, как мы помним, для многих аналитических задач (например, расчета расстояний, площадей) предпочтительно использовать систему координат в метрических единицах.


Проверим текущую систему координат наших данных


In [ ]:
current_crs = points.crs
print("Текущая система координат:", current_crs)


Оценим подходящую UTM-зону для наших точек и перепроецируем в нее


In [ ]:
utm_crs = points.estimate_utm_crs()       # автоматически определяем UTM-зону
points_utm = points.to_crs(utm_crs)       # преобразуем GeoSeries в эту проекцию
print("Новая система координат:", points_utm.crs)

Теперь обе точки заданы в спроецированной системе координат (UTM Zone, в метрах). Вычислим расстояние между ними.

Сначала попробуем вычислить расстояние напрямую в географической CRS (в градусах), а затем в метрической CRS:


In [ ]:
# Расстояние между точками в градусах (как есть, в исходной CRS)
dist_deg = points.geometry[0].distance(points.geometry[1])
# Расстояние между точками в метрах (после проекции в UTM)
dist_m = points_utm.geometry[0].distance(points_utm.geometry[1])
print(f"Расстояние в градусах: {dist_deg:.6f}°")
print(f"Расстояние в метрах: ~{dist_m:.0f} м")

В дальнейшем, **всегда перед измерением расстояний или площадей, убедимся, что данные заданы в метрической системе координат**.


### 1.2 Буферные зоны (Buffer)


**Буферная зона** – это область вокруг геометрического объекта на заданном расстоянии. Чаще всего буфер строят вокруг точек, линий или полигонов, чтобы выделить зону влияния или ближайшего окружения.

**При работе с буферами важно помнить про систему координат**: в географической системе координат (широта/долгота) радиус буфера указывается в градусах. Поэтому для измерения реальных расстояний буфер можно строить после в метрических системах координат (например, UTM).


In [ ]:
buffer_500 = points_utm.buffer(500)

buffer_500.explore(tiles='CartoDB positron')

Построение буферных зон — относительно простая операция, но она регулярно применяется в пространственном анализе. При ее выполнении особенно важно внимательно отслеживать единицы измерения в используемой системе координат.


### 1.3 Пространственное объединение (Spatial Join)


**Пространственное объединение (spatial join)** — операция, при которой к каждому объекту одного слоя присоединяются атрибуты объекта другого слоя, с которым он связан пространственно. Это может быть принадлежность к району, попадание в буфер и т.д.

Однако само по себе пространственное объединение — это не только механическое добавление атрибутов на основе пересечений геометрий. Чтобы этот процесс был действительно точным и осмысленным, важно понимать, каким именно образом соотносятся объекты в пространстве. Пересекаются ли они? Один внутри другого? Или, быть может, лишь касаются границ?
Для того чтобы корректно интерпретировать такие случаи, используются пространственные предикаты — ключевой инструмент в пространственном анализе, позволяющий задать и проверить топологические отношения между геометриями.

**Пространственные предикаты** – это функции, позволяющие определить топологическое отношение между двумя геометрическими объектами (точками, линиями, полигонами и т.д.). Предикаты позволяют отвечать на вопросы вроде: «Пересекаются ли эти объекты?», «Находится ли один объект внутри другого?», «Касаются ли они границ друг друга?» и т.п. Знание пространственных отношений помогает выполнять пространственные запросы и объединения данных (spatial join), фильтрацию на основе взаимного расположения и др.

Ниже приведена таблица основных бинарных предикатов (отношений) с кратким описанием:

| **Предикат**                     | **Описание** (условие истинности)                                                                                                                                                                                                                                                                 |
| -------------------------------- | ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **intersects** (_пересекается_)  | **Имеют общую точку** – Геометрии пересекаются, если они имеют хотя бы одну общую точку в пространстве (любое наложение границ или внутренних областей). Эквивалентно логическому отрицанию disjoint (см. ниже).                                                                                  |
| **disjoint** (_не пересекается_) | **Не имеют общих точек** – Геометрии не пересекаются, если у них **нет ни одной** общей точки. То есть полностью разделены в пространстве.                                                                                                                                                        |
| **within** (_внутри_)            | **A внутри B** – Геометрия A находится целиком внутри геометрии B, не выходя за ее границы. Все точки A (и граница, и внутренняя часть) лежат во внутренней области B. (Эквивалентно B.contains(A)).                                                                                              |
| **contains** (_содержит_)        | **A содержит B** – Геометрия A полностью содержит геометрию B. То есть ни одна часть B не лежит вне A, при этом B не просто граничит с A, а имеет хотя бы одну точку строго внутри A. (Эквивалентно B.within(A)).                                                                                 |
| **touches** (_касается_)         | **Касание границ** – Геометрии имеют хотя бы одну общую точку на границе, но **не** пересекаются по внутренней области. Иными словами, их границы соприкасаются, но пересечения интерьеров нет.                                                                                                   |
| **overlaps** (_перекрывает_)     | **Частичное перекрытие** – Геометрии частично перекрываются, если они имеют общую часть, но ни одна не содержит другую целиком. При этом геометрии одного типа (полигон-полигон или линия-линия) – например, два полигона, которые пересекаются по площади, но каждый выходит за пределы другого. |
| **crosses** (_перекрещивается_)  | **Пересечение с выходом** – Геометрии пересекаются _внутренними точками_ таким образом, что пересечение имеет меньшую размерность. Например, линия пересекает полигон, входя и выходя из него (часть линии внутри, часть снаружи), или две линии пересекаются в одной точке.                      |


Какой предикат нам понадобится для пространствнного объединения точек метро с полигонами районов города?

(ответ будет ниже по ходу работы)


Перейдем к задаче – определим, в каких районах города находятся станции метро


In [ ]:
#Прочитаем данные о станциях метро
metro = gpd.read_file('data/spb_metro.geojson')

metro.explore(tiles='CartoDB positron')

In [ ]:
#Прочитаем данные о городских районах
admin_district = gpd.read_file('data/spb_admin.gpkg', layer="district")

admin_district.explore(tiles='CartoDB positron')

C помощью пространственного объединения опрееделим, какая станция находится в каком районе


In [ ]:
metro_with_district = gpd.sjoin(metro, admin_district, predicate='within')
metro_with_district[['name', 'geometry', 'NAME']].head()

Посчитаем количество станций в каждом из районов


In [ ]:
count_by_district = metro_with_district.groupby("NAME")["name"].count().reset_index()
count_by_district.columns = ["Район", "Количество станций"]
count_by_district

Понимание пространственных предикатов особенно важно не только для объединения данных, но и для выполнения **фильтрации объектов на основе их взаимного расположения**. Иногда нам вовсе не нужно присоединять атрибуты одного слоя к другому — достаточно отобрать те объекты, которые, например, находятся в пределах заданной области, пересекаются с буферной зоной или касаются границ интересующей территории.
Такая фильтрация позволяет гибко и точно выделять пространственно значимые подмножества данных, что делает её неотъемлемым инструментом пространственного анализа.


### 1.4 Фильтрация объектов по пространственному признаку


Одно из распространённых действий в пространственном анализе – отбор объектов, удовлетворяющих определённому пространственному условию. Например, можно захотеть **выбрать все точки (объекты) внутри заданного полигона**. Это называется пространственной фильтрацией (фильтрация по местоположению).

Давайте отфильтруем все кафе Санкт-Петербурга, которые находятся в зоне 500-метровой доступности от станций метро


Подготовим данные


In [ ]:
#Прочитаем данные о станциях метро
metro = gpd.read_file('data/spb_metro.geojson')

Построим буферы 500-метровой доступности вокруг станций (предварительно проверим систему координат и перепроецируем, если нужно)


In [ ]:
metro.crs

Данные в географической системе координат, нам это не подходит, поэтому перепроецируем их перед тем как строить буферы


In [ ]:
utm_crs = metro.estimate_utm_crs()   # автоматически определяем UTM-зону
metro_utm = metro.to_crs(utm_crs)  

Построим буферы


In [ ]:
metro_buff_500 = metro_utm.buffer(500)
metro_buff_500.explore(tiles='CartoDB positron')

Выгрузим информацию о кафе из OSM, отфильтруем только точки и перепроецируем


In [ ]:
tags = {"amenity": "cafe"}
cafes = ox.features_from_place("Санкт-Петербург, Россия", tags)

#фильтрация точек
cafes = cafes[cafes.geometry.type == 'Point']

#перепроецирование
cafes_utm = cafes.to_crs(utm_crs) 


cafes_utm.explore(tiles='CartoDB positron')

Теперь отфильтруем только те кафе, которые строго находятся **внутри** 500-метровых буферов вокруг станций метро


In [ ]:
# Создаем один общий буфер (объединение всех зон)
combined_buffer = metro_buff_500.union_all()

# Фильтруем кафе внутри общего буфера
cafes_within = cafes_utm[cafes_utm.geometry.within(combined_buffer)]

In [ ]:
cafes_within.explore(tiles='CartoDB positron')

Мы рассмотрели ещё один важный приём пространственного анализа – фильтрацию объектов по принадлежности области. Он опирается на предикаты пространственных отношений (`within`, `contains`, и др.) и позволяет извлекать информацию об объектах в заданных границах. Это очень мощный инструмент, который используется в самых разных задачах ГИС, от простых запросов («найти все объекты внутри...») до сложных географических вычислений.


## 2. Агрегирование данных


Одним из способов оценить **плотность точечных объектов** на территории является разделение области на сетку (fishnet) и подсчет количества точек в каждой ячейке. Такой подход дает наглядную картину распределения объектов по пространству.

В этом разделе мы посчитаем плотность размещения кафе в Центральном районе Санкт-Петербурга. Для этого:

- Построим регулярную сетку квадратных ячеек, покрывающую интересующую нас территорию.
- Выполним пространственное объединение: определим, сколько кафе попадает в каждую ячейку.
- Вычислим плотность кафе (количество на км²) по ячейкам и визуализируем результат.
- Дополнительно покажем, как можно агрегировать другие атрибуты (например, средний рейтинг кафе) по пространственным ячейкам.


### 2.1 Построение квадратной сетки (fishnet)

Чтобы создать сетку, нам нужны границы рассматриваемой территории и выбор размера ячейки. Воспользуемся границами одного из районов Санкт-Петербурга и зададим размер квадрата, например 200 метров.

Напишем алгоритм, который по экстенту заданной области и размеру стороны квадрата генерирует GeoDataFrame с полигональной сеткой, покрывающей эту область:


##### Шаг 0 - Получение данных о границах территории


In [ ]:
#Запишем в переменную название изучаемого района 
area = "Центральный район, Санкт-Петербург"

# Получаем границу района с помощью номинального геокодирования
admin_polygon = ox.geocode_to_gdf(area)
print("Исходная CRS:", admin_polygon.crs)


Если набор в географических координатах, перепроецируем в UTM


##### Шаг 1 - Перепроецирование данных в метрическую систему координат


In [ ]:
# Определяем и устанавливаем UTM-проекцию
utm_zone = admin_polygon.estimate_utm_crs()
admin_polygon = admin_polygon.to_crs(utm_zone)
print("Проекция для сетки:", admin_polygon.crs)

In [ ]:
# Визуализация
ax = admin_polygon.plot(marker='o', color='green', figsize=(6,6))
plt.title("Границы района (CRS: {})".format(utm_crs))
plt.show()

##### Шаг 2 - Определение экстента (bounding box)


Определим границы области для построения сетки


In [ ]:
# Определяем минимальные и максимальные значения координат по долготе и по широте
minx, miny, maxx, maxy = admin_polygon.total_bounds

# Строим полигон-рамку (для визуалиации того, что мы получили - не обязательный шаг непосредственно для пострения сетки)
bbox = Polygon([
    (minx, miny), (maxx, miny),
    (maxx, maxy), (minx, maxy)
])

# Посмотрим на результат
ax = admin_polygon.plot(marker='o', color='green', figsize=(6,6))
gpd.GeoSeries([bbox], crs=utm_crs).plot(
    ax=ax, facecolor='none', edgecolor='red', linewidth=2
)
plt.title("Граница области (bounding box)")
plt.show()

##### Шаг 3 - Определеним размер ячейки


In [ ]:
# Параметр: размер ячейки (в метрах)
square_size = 500

##### Шаг 4 - Сгенерируем ячейки внутри заданного ограничивающего полигона


In [ ]:
# Список будущих полигонов
grid_cells = []

# Стартовые координаты
x0, y0 = minx, miny
x, y = x0, y0

# Проходим по строкам и столбцам
while y <= maxy:
    while x <= maxx:
        grid_cells.append(
            box(x, y, x + square_size, y + square_size)
        )
        x += square_size
    # после строки: сброс по X и шаг по Y
    x = x0
    y += square_size


# Отфильтруем первые 10 ячеек, чтобы на них посмотреть 
first_few = grid_cells[:10]
ax = gpd.GeoSeries(first_few, crs=utm_crs).plot(
    figsize=(6,6), edgecolor='blue', facecolor='none', zorder=3
)

# отобразим также границу района
admin_polygon.plot(ax=ax, marker='o', color='green', linewidth=1)

# и его экстент
gpd.GeoSeries([bbox], crs=utm_crs).plot(
    ax=ax, facecolor='none', edgecolor='red', linewidth=2
)


plt.title("Первые 10 клеток + границы (admin & bbox)")
plt.show()

##### Шаг 5 - Собирем итоговую сетку в GeoDataFrame


In [ ]:
# Собираем всё в GeoDataFrame
fishnet = gpd.GeoDataFrame({'geometry': grid_cells}, crs=utm_crs)

# Показываем итоговую сетку поверх данных
ax = admin_polygon.plot(marker='o', color='green', figsize=(6,6))
fishnet.plot(ax=ax, edgecolor='gray', linewidth=0.5, alpha=0.7)
plt.title("Итоговая регулярная сетка (fishnet/regular grid)")
plt.show()

##### Шаг 6 - Напишем функцию на основе алгоритма


На основе нашего алгоритма напишем функцию, которая на вход будет принимать только исходные данные (для определения экстента) и шаг сетки


In [ ]:
from shapely.geometry import Polygon

def create_regular_grid(gdf, square_size):
    # Если набор в географических координатах, перепроецируем в UTM
    utm_crs = gdf.estimate_utm_crs()
    gdf_utm = gdf.to_crs(utm_crs)
    # Получаем границы (bounding box) области
    minX, minY, maxX, maxY = gdf_utm.total_bounds
    grid_cells = []
    x, y = minX, minY
    # Двойной цикл: заполняем прямоугольник границ квадратами заданного размера
    while y <= maxY:
        while x <= maxX:
            grid_cells.append(box(x, y, x + square_size, y + square_size))
            x += square_size # двигаемся по X
        x = minX             # возврат к началу по X
        y += square_size     # двигаемся по Y (следующий ряд)
    # Формируем GeoDataFrame
    fishnet = gpd.GeoDataFrame(geometry=grid_cells, crs=utm_crs)
    return fishnet



In [ ]:
# Строим сетку с шагом 1000 м
fishnet = create_regular_grid(admin_district, square_size=1000)
print(f"Создано ячеек: {len(fishnet)}")
fishnet.head(3)

Посмотрим на сетку на карте


In [ ]:
fishnet.explore(tiles='CartoDB positron')

Отфильтруем те ячейки, которые находятся внутри границ Санкт-Петербурга


In [ ]:
fishnet_filtered = fishnet[fishnet.intersects(admin_district.to_crs(cafes_utm.crs).union_all())]

In [ ]:
fishnet_filtered.explore(tiles='CartoDB positron')

### 2.2 Агрегируем данные - подсчет количества объектов в ячейках (плотность кафе)

Теперь получим данные о расположении **кафе** в исследуемом районе. Используем OSMnx для выгрузки точек с тегом `amenity=cafe` в границах района. Затем с помощью spatial join определим, сколько кафе попадает в каждую ячейку сетки.


##### Шаг1: Выполняем пространственное объединение кафе и сетки(Spatial Join)


Пересекаем точки с кафе и полигоны регулярной сетки


In [ ]:
# sjoin: определяем, в какой квадрат попадает каждое кафе
cafes_in_grid = gpd.sjoin(cafes_utm, fishnet_filtered, predicate="within", how="left")

cafes_in_grid.head()


Считаем количество точек попавших в каждый из полигонов


In [ ]:
# Подсчитываем число точек в каждой ячейке (группируем по индексу ячейки)
cafes_count_per_cell = cafes_in_grid.groupby('index_right').size()

cafes_count_per_cell.head()

Записываем данные в исходный слой с сеткой


In [ ]:
# Добавляем столбец с количеством кафе в соответствующую ячейку сетки
fishnet_filtered['cafes_count'] = cafes_count_per_cell
fishnet_filtered['cafes_count'] = fishnet_filtered['cafes_count'].fillna(0).astype(int)  # заполняем 0, где кафе нет
fishnet_filtered.head(3)

Смотрим на результат


In [ ]:
fishnet_filtered.explore(column='cafes_count', cmap='YlGnBu', tiles='CartoDB positron')

##### Шаг2: Считаем плотность объектов


Мы добавили к каждой ячейке количество кафе `cafes_count`. Теперь рассчитаем **плотность кафе** – сколько кафе приходится на квадратный километр в границах каждой ячейки


In [ ]:
# Вычисляем площадь каждой ячейки в квадратных метрах
cell_area_m2 = fishnet_filtered.geometry.area  
# Добавляем столбец плотности кафе (число кафе на 1 км²)
fishnet_filtered['cafe_density'] = fishnet_filtered['cafes_count'] / (cell_area_m2 / 1_000_000)

fishnet_filtered.head()


Каждой ячейке присвоим цвет в зависимости от плотности кафе:


In [ ]:
fishnet_filtered.explore(column='cafe_density', cmap='YlGnBu', scheme='NaturalBreaks', legend=True,
legend_kwds={'caption': 'Кафе на км²'}, tiles='CartoDB positron')


## 3. Агрегирование атрибутивной информации


Прочитаем данные об МКД в Санкт-Петербурге и о границах районов


In [ ]:
#Данные с округами в Санкт-Петербурге в формате geopackage
admin_okrug = gpd.read_file('data/spb_admin.gpkg', layer="okrug")

#МКД Санкт-Петербурга в формате csv
mkd = pd.read_csv('data/spb_mkd.csv')
#Разбиваем поле с координатами на два - 'lat' -- широта; 'lon' -- долгота
mkd[['lat', 'lon']] = mkd['coordinates'].str.split(',', expand=True).astype(float)
#На основе DataFrame создаем GeoDataFrame, определяя геометрию на основе координат
mkd_gdf = gpd.GeoDataFrame(mkd, geometry=gpd.points_from_xy(mkd['lon'], mkd['lat']), crs=4326)

Давайте посмотрим, какая атрибутивная информация есть в слое с МКД


In [ ]:
# такая запись помогает нам не просто показать первые строки, но и транспонировать таблицу, чтобы увидеть сразу все столбцы
mkd_gdf.head().T

У нас, к сожалению, нет документации с описанием названий полей, но мы +- можем догадываться, какая именно информация содержится в полях.

Из интересной информации, которую бы мы хотели посмотреть по каждому из округов можно выбрать:
data_buildingdate - год постройки;

**Давайте для каждого района посчитаем средний, медианный, самый ранний и самый поздний год постройки МКД**


Первые шаги будут аналогичны тем, которые мы выполняли на прошлой неделе при подсчете кол-ва точек по ячейкам регулярной сетки


##### Шаг1: Выполняем пространственное объединение (Spatial Join)

Пересекаем точки с кафе и полигоны регулярной сетки (не забываем, что слои должны быть в одной системе координат)


In [ ]:
# sjoin: определяем, в какой округ попадает каждый из МКД
mkd_in_okrug = gpd.sjoin(mkd_gdf, admin_okrug, predicate="within", how="left")

mkd_in_okrug.head()

В атрибутивной таблице выше - все поля, которые идут после "geometry" -- атрибутивная информация из слоя с округами, в зависимости от того, в какой округ попадает та или иная точка


##### Шаг2: Рассчитываем статистику


Группируем данные на основе их попадания в тот или иной округ (например, по полю index_right) и рассчитываем значения

**Важно**: поскольку мы хотим рассчитывать количественную статистику, необходимо проверить типы данных полей
(data_buildingdate, data_residents) и убедиться, что они имеют числовой тип или преобразовать их в числовой


In [ ]:
# 1. Посмотреть исходные типы
print("До преобразования:")
print(mkd_in_okrug[['data_buildingdate']].dtypes)

# 2. Преобразование

mkd_in_okrug['data_buildingdate'] = pd.to_numeric(mkd_in_okrug['data_buildingdate'], errors='coerce')

# 3. Проверка — типы после
print("\nПосле преобразования:")
print(mkd_in_okrug['data_buildingdate'].dtypes)


Отлично! Теперь мы точно можем рассчитывать количественную статистику на основе этого поля


Рассчитаем средний год постройки МКД в разных округах


In [ ]:
build_year_mean = mkd_in_okrug.groupby('index_right')['data_buildingdate'].mean()

# Добавляем в данные поля с средним годом постройки в слой с округами
admin_okrug['build_year_mean'] = build_year_mean

# Смотрим на результат
admin_okrug.explore(column='build_year_mean', cmap='YlGnBu', tiles='CartoDB positron')



Выглядит очень интересно и правдоподобно :)


По аналогии посчитаем медианный год постройки, а также самый ранний и самый поздний


In [ ]:
build_year_median = mkd_in_okrug.groupby('index_right')['data_buildingdate'].median()
build_year_min = mkd_in_okrug.groupby('index_right')['data_buildingdate'].min()
build_year_max = mkd_in_okrug.groupby('index_right')['data_buildingdate'].max()

# Добавляем в данные соответсвующие поля
admin_okrug['build_year_median'] = build_year_median
admin_okrug['build_year_min'] = build_year_min
admin_okrug['build_year_max'] = build_year_max



Отобразим 4 показателя вместе для сравнения


In [ ]:
metrics = [
    ('build_year_mean', 'Средний год постройки'),
    ('build_year_median', 'Медианный год постройки'),
    ('build_year_min', 'Самый ранний год постройки'),
    ('build_year_max', 'Самый поздний год постройки'),
]

# Общий диапазон для всех карт
vmin = admin_okrug[[m for m, _ in metrics]].min().min()
vmax = admin_okrug[[m for m, _ in metrics]].max().max()

# Создаём одну фигуру с 1 строкой и 4 столбцами
fig, axes = plt.subplots(1, 4, figsize=(20, 5), constrained_layout=True)

# Создаём общий нормализатор и мэппер для цветовой шкалы
norm = mpl.colors.Normalize(vmin=vmin, vmax=vmax)
cmap = mpl.cm.viridis  # или любая другая

for ax, (metric, title) in zip(axes, metrics):
    admin_okrug.plot(
        column=metric,
        ax=ax,
        cmap=cmap,
        norm=norm,
        linewidth=0.5,
        edgecolor='gray'
    )
    ax.set_title(title, fontsize=12)
    ax.axis('off')

# Общая colorbar
sm = mpl.cm.ScalarMappable(cmap=cmap, norm=norm)
sm._A = []  # пустой массив, чтобы colorbar работал
cbar = fig.colorbar(sm, ax=axes.tolist(), orientation='horizontal',
                    fraction=0.05, pad=0.02)
cbar.set_label('Год постройки', fontsize=12)

plt.show()

Красота!


## 4. Итог


На этой неделе мы познакомились с ключевыми инструментами пространственной аналитики: научились создавать буферные зоны, обсудили пространственные предикаты и пространственное объединение, а также научились создавать регулярную сетку и агрегировать по ней точечные данные.
